# Basic Bathtub Scenario Analysis

Understanding the model dynamics before policy optimisation.

## Model Overview

The basic bathtub scenario models:
- **Subjects** with a durability feature (log-normal distribution)
- **Failure** governed by a bathtub-shaped hazard (CompoundWeibull)
- **Service** that reduces effective age
- **Economics**: revenue per time unit, service cost, failure cost

The goal is to find service policies that maximise net value.

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate, optimize

from src.scenarios import BasicBathtubScenario
from src.survival import CompoundWeibull, Weibull

## Scenario Parameters

In [ ]:
# Parameters for analysis (same as optimisation notebook)
scenario = BasicBathtubScenario(
    scale1=100.0,
    scale2=200.0,
    service_cost=0.5,
    failure_cost=150.0,
    revenue_per_time=1.50,
)

fm = scenario.failure_model
print("Bathtub parameters:")
print(f"  Early component:  shape1={fm.shape1}, scale1={fm.scale1}")
print(f"  Wear-out:         shape2={fm.shape2}, scale2={fm.scale2}")
print(f"  Service effect:   delta_t={fm.delta_t} (effective age reduction per service)")
print()
print("Economics:")
print(f"  Revenue per time: {scenario.costs.revenue_per_time}")
print(f"  Service cost:     {scenario.costs.service_cost}")
print(f"  Failure cost:     {scenario.costs.failure_cost}")
print()
print("Durability distribution:")
print(f"  Log-normal with mean={scenario.durability_mean}, std={scenario.durability_std}")

## Durability Distribution

In [ ]:
subjects = scenario.generate_subjects(10000, seed=42)
durabilities = [s.get_feature('durability') for s in subjects]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histogram
axes[0].hist(durabilities, bins=50, density=True, alpha=0.7, edgecolor='black')
axes[0].axvline(np.median(durabilities), color='red', linestyle='--', label=f'Median: {np.median(durabilities):.2f}')
axes[0].axvline(np.mean(durabilities), color='orange', linestyle='--', label=f'Mean: {np.mean(durabilities):.2f}')
axes[0].set_xlabel('Durability')
axes[0].set_ylabel('Density')
axes[0].set_title('Durability Distribution (Log-Normal)')
axes[0].legend()

# Percentiles table
percentiles = [5, 10, 20, 50, 80, 90, 95]
pct_values = [np.percentile(durabilities, p) for p in percentiles]
axes[1].axis('off')
table_data = [[f'{p}th' for p in percentiles], [f'{v:.3f}' for v in pct_values]]
table = axes[1].table(cellText=table_data, rowLabels=['Percentile', 'Durability'],
                       loc='center', cellLoc='center')
table.scale(1, 2)
axes[1].set_title('Durability Percentiles', y=0.7)

plt.tight_layout()
plt.show()

## Hazard Function

The hazard is a **CompoundWeibull** (sum of two Weibull hazards):

$$h(t) = h_1(t) + h_2(t)$$

Where each Weibull hazard is:

$$h(t; k, \lambda) = \frac{k}{\lambda} \left(\frac{t}{\lambda}\right)^{k-1}$$

**Durability scales the scale parameters:**

$$\lambda_i \rightarrow \lambda_i \times d$$

This gives:

$$h(t; d) = h_1(t) \times d^{-k_1} + h_2(t) \times d^{-k_2}$$

In [ ]:
# Hazard curves for different durability percentiles
percentiles_to_plot = [20, 50, 80]
dur_values = {p: np.percentile(durabilities, p) for p in percentiles_to_plot}

t_values = np.linspace(0.5, 200, 300)

plt.figure(figsize=(10, 6))

for p, d in dur_values.items():
    bathtub = CompoundWeibull(
        shape1=fm.shape1, scale1=fm.scale1 * d,
        shape2=fm.shape2, scale2=fm.scale2 * d
    )
    hazards = [bathtub.hazard(t) for t in t_values]
    
    # Find minimum
    result = optimize.minimize_scalar(bathtub.hazard, bounds=(0.1, 300), method='bounded')
    t_min, h_min = result.x, result.fun
    
    plt.plot(t_values, hazards, label=f'{p}th %ile (d={d:.2f})', linewidth=2)
    plt.scatter([t_min], [h_min], s=80, zorder=5, marker='o')

plt.xlabel('Time (effective age)', fontsize=12)
plt.ylabel('Hazard h(t)', fontsize=12)
plt.title(f'Bathtub Hazard by Durability\n'
          f'shape₁={fm.shape1} (infant mortality), shape₂={fm.shape2} (wear-out)\n'
          f'Dots mark minimum hazard',
          fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, 200)
plt.tight_layout()
plt.show()

print("Observations:")
print(f"- shape₁={fm.shape1} < 1 creates HIGH early hazard (infant mortality)")
print(f"- shape₂={fm.shape2} > 1 creates INCREASING late hazard (wear-out)")
print("- True bathtub shape: high → minimum → high")
print("- Higher durability → lower hazard and later minimum")

## Hazard Components

Breaking down the two Weibull components:

In [ ]:
# Show the two components separately for median durability
d_median = dur_values[50]

weibull1 = Weibull(shape=fm.shape1, scale=fm.scale1 * d_median)
weibull2 = Weibull(shape=fm.shape2, scale=fm.scale2 * d_median)
bathtub = CompoundWeibull(fm.shape1, fm.scale1 * d_median, fm.shape2, fm.scale2 * d_median)

h1 = [weibull1.hazard(t) for t in t_values]
h2 = [weibull2.hazard(t) for t in t_values]
h_total = [bathtub.hazard(t) for t in t_values]

plt.figure(figsize=(10, 6))
plt.plot(t_values, h1, '--', label=f'h₁(t): shape={fm.shape1} (infant mortality)', linewidth=2)
plt.plot(t_values, h2, '--', label=f'h₂(t): shape={fm.shape2} (wear-out)', linewidth=2)
plt.plot(t_values, h_total, 'k-', label='h(t) = h₁ + h₂ (total)', linewidth=2)

plt.xlabel('Time (effective age)', fontsize=12)
plt.ylabel('Hazard h(t)', fontsize=12)
plt.title(f'Hazard Components (median durability d={d_median:.2f})', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, 200)
plt.tight_layout()
plt.show()

print(f"h₁ (infant mortality): DECREASING from high at t=0")
print(f"h₂ (wear-out): INCREASING, dominates at later times")
print(f"Total: bathtub shape with minimum where components cross")

## Minimum Hazard Analysis

For each durability level:
- Where is hazard minimal?
- What would lifetime/economics be at that constant hazard?

In [ ]:
percentiles_analysis = [20, 50, 80]
dur_analysis = {p: np.percentile(durabilities, p) for p in percentiles_analysis}

print(f"{'%ile':<6} {'Durability':>10} {'t_min':>8} {'h_min':>10} {'E[T] @ h_min':>14}")
print("-" * 55)

min_hazard_data = {}
for p, d in dur_analysis.items():
    bathtub = CompoundWeibull(fm.shape1, fm.scale1 * d, fm.shape2, fm.scale2 * d)
    
    # Find minimum hazard
    result = optimize.minimize_scalar(bathtub.hazard, bounds=(0.1, 300), method='bounded')
    t_min, h_min = result.x, result.fun
    
    # E[T] if hazard were constant at h_min
    E_T_const = 1 / h_min
    
    min_hazard_data[p] = (d, t_min, h_min, E_T_const)
    print(f"{p:>4}th {d:>10.3f} {t_min:>8.1f} {h_min:>10.5f} {E_T_const:>14.1f}")

print("\nThe minimum hazard occurs around t=50-80 depending on durability.")
print("If we could keep subjects at this hazard, expected lifetime would be ~100-150.")

## Expected Lifetime and Economics (No Service)

Actual outcomes following the full bathtub curve:

In [ ]:
MAX_TIME = 150.0

print(f"{'%ile':<6} {'Durability':>10} {'E[T] actual':>12} {'E[T] ideal':>12} {'Net (actual)':>12} {'Gain possible':>14}")
print("-" * 75)

results = []
for p, d in dur_analysis.items():
    bathtub = CompoundWeibull(fm.shape1, fm.scale1 * d, fm.shape2, fm.scale2 * d)
    
    # Actual expected lifetime
    E_T_actual, _ = integrate.quad(bathtub.survival, 0, 500)
    
    # Ideal (at minimum hazard)
    _, t_min, h_min, E_T_ideal = min_hazard_data[p]
    
    # Economics
    revenue_actual = E_T_actual * scenario.costs.revenue_per_time
    revenue_ideal = E_T_ideal * scenario.costs.revenue_per_time
    net_actual = revenue_actual - scenario.costs.failure_cost
    net_ideal = revenue_ideal - scenario.costs.failure_cost
    gain = net_ideal - net_actual
    
    results.append((p, d, E_T_actual, E_T_ideal, net_actual, gain))
    print(f"{p:>4}th {d:>10.3f} {E_T_actual:>12.1f} {E_T_ideal:>12.1f} {net_actual:>12.1f} {gain:>14.1f}")

print("\n'Gain possible' = value we could extract if we kept subjects at minimum hazard.")
print("This is the budget for service costs.")

In [ ]:
# Visualise
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

pcts = [r[0] for r in results]
E_T_actual = [r[2] for r in results]
E_T_ideal = [r[3] for r in results]
nets = [r[4] for r in results]
gains = [r[5] for r in results]

x = np.arange(len(pcts))
width = 0.35

axes[0].bar(x - width/2, E_T_actual, width, label='Actual (bathtub)', color='steelblue', alpha=0.7)
axes[0].bar(x + width/2, E_T_ideal, width, label='Ideal (const h_min)', color='green', alpha=0.7)
axes[0].set_xlabel('Durability Percentile')
axes[0].set_ylabel('Expected Lifetime')
axes[0].set_title('Expected Lifetime: Actual vs Ideal')
axes[0].set_xticks(x)
axes[0].set_xticklabels([f'{p}th' for p in pcts])
axes[0].legend()

colors = ['green' if n > 0 else 'red' for n in nets]
axes[1].bar([f'{p}th' for p in pcts], nets, color=colors, alpha=0.7, label='Net (actual)')
axes[1].bar([f'{p}th' for p in pcts], gains, bottom=nets, color='lightgreen', alpha=0.7, label='Gain possible')
axes[1].set_xlabel('Durability Percentile')
axes[1].set_ylabel('Net Value')
axes[1].set_title('Net Value and Potential Gain from Service')
axes[1].axhline(0, color='black', linewidth=0.5)
axes[1].legend()

plt.tight_layout()
plt.show()

## Effect of Service on Effective Age

Service reduces **effective age** by `delta_t`:

```
effective_age = time - service_count × delta_t
```

With `interval` = time between services:
- If interval = delta_t: effective_age oscillates between 0 and interval
- If interval > delta_t: effective_age drifts upward (net +interval-delta_t per cycle)

In [ ]:
delta_t = fm.delta_t

# Different interval/delta_t combinations
scenarios_eff = [
    ("No service", np.inf, delta_t),
    (f"interval=25, δt={delta_t} (drift +10/cycle)", 25, delta_t),
    (f"interval={delta_t}, δt={delta_t} (stable)", delta_t, delta_t),
]

# Find minimum hazard region for median durability
d_med = dur_values[50]
bathtub_med = CompoundWeibull(fm.shape1, fm.scale1 * d_med, fm.shape2, fm.scale2 * d_med)
res = optimize.minimize_scalar(bathtub_med.hazard, bounds=(0.1, 300), method='bounded')
t_min_med = res.x

plt.figure(figsize=(12, 6))

for label, interval, dt in scenarios_eff:
    times = np.arange(0, 151)
    if interval == np.inf:
        eff_ages = times.astype(float)
    else:
        service_counts = times // interval
        eff_ages = np.maximum(0, times - service_counts * dt).astype(float)
    
    plt.plot(times, eff_ages, label=label, linewidth=2)

# Mark minimum hazard region
plt.axhspan(t_min_med - 10, t_min_med + 10, alpha=0.2, color='green', 
            label=f'Min hazard region (~{t_min_med:.0f})')

plt.xlabel('Real Time', fontsize=12)
plt.ylabel('Effective Age', fontsize=12)
plt.title(f'Effective Age Trajectories (delta_t={delta_t})', fontsize=12)
plt.legend(fontsize=10, loc='upper left')
plt.grid(True, alpha=0.3)
plt.xlim(0, 150)
plt.ylim(0, 160)
plt.tight_layout()
plt.show()

print(f"Minimum hazard is around effective_age ≈ {t_min_med:.0f}")
print(f"With current delta_t={delta_t}:")
print(f"  - interval=25 drifts up, eventually reaching minimum region")
print(f"  - interval={delta_t} stays in 0-{delta_t} range (infant mortality zone)")

## Summary

**Model setup:**
- Subjects have durability (log-normal), affecting hazard via scale
- Hazard = infant mortality (decreasing) + wear-out (increasing) = bathtub shape
- Service reduces effective age by delta_t

**Key dynamics:**
- Without service, most subjects fail and are unprofitable
- Minimum hazard is around effective_age ≈ 50-80
- Service can slow aging but current delta_t doesn't keep subjects at minimum
- Trade-off: frequent service stays in infant mortality, infrequent reaches wear-out

See `policy_optimisation.ipynb` for finding optimal service policies.